In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# $k$ nearest neighbours for regression

The [$k$ nearest neighbours](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) method is one of the simplest machine learning methods, yet it often performs incredibly well.

The algorithm is as follows:
1. Given a new datapoint $x_i$, find the $k$ closest datapoints in the dataset $X$.
2. Estimate an output value as the mean of the outputs of the $k$ closest datapoints.

The number of neighbours, $k$, to look at typically lies in the range of $[1, \dots, 30]$. The question is how to select the right $k$ for a given task. One way to do that is by using *cross validation*.

## Cross validation
[Cross validation](https://en.wikipedia.org/wiki/Cross-validation_(statistics)) (CV) is used both for testing the performance of a method and for selecting hyperparameters, such as the number of nearest neighbours, $k$. The most common version, $K$-fold CV, divides the dataset into $K$ partitions and uses $K-1$ as the training set and the remaining partition as test set. This is repeated $K$ times, each time using a different partition as test set. It is *very* important to randomise the dataset before partitioning it, in case the data points have been sorted.

Typical values of $K$ are 3, 5, and 10. The higher the better, but also more time is required to run the CV, so in practice the value is chosen as a trade-off between statistical significance and time constraints.



## Exercises
1. Implement $k$ nearest neighbours, but for now just fix the value of $k$ manually.
2. Implement cross validation.
3. Use cross validation to find the best $k$ for the dataset.

Test the implemented methods on the data in `ex1.dat`, `ex2.dat`, and `ex3.dat`.


## Data

Load data from exercise 1.

In [ ]:
data = np.loadtxt("../data/ex1.dat")
X = data[:, 0]
y = data[:, 1]

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X, y)
plt.show()

## $k$ nearest neighbours

Let's say we want to predict the value at $x_0 = 2$. This means we need to look for nearest neighbours around this position:

In [ ]:
# Position to look for nearest neighbours at:
x0 = 2

fig, ax = plt.subplots()
ax.scatter(X, y)
ax.axvline(x0, color="C1")
plt.show()

We will predict the value at $x_0=2$ based on the $k=3$ nearest neighbours. First, we need to locate these, then compute the mean of their output values.

In [ ]:
k = 3    # number of neighbours to look for

# Compute the distance from all points to x0
dists = np.abs(X - x0)

# Find the indices of the k closest:
idx = np.argsort(dists)[:k]
print("k = {} closest data points: x = {}".format(k, X[idx]))

# Find mean of the outputs:
prediction = np.mean(y[idx])
print("Prediction for x0 = {}: y0 = {}".format(x0, prediction))


Let's highlight the found points and our prediction:

In [ ]:
fig, ax = plt.subplots()

# Plot the data, but change the colour if the point is one of the nearest neighbours:
ax.scatter(X, y, color=["C1" if i in idx else "C0" for i in range(len(X))])

# Plot our prediction:
ax.scatter(x0, prediction, s=60, marker="x", color="C1")

ax.axvline(x0, color="C1")

plt.show()

### $k$ nearest neighbours as a function

Let's wrap the above in an easy-to-use function that we can use on an entire list of points to make predictions at:

In [ ]:
def knn(k, xf, X, y):
    """
    Compute the k nearest neighbours' estimate for all x's in `xf`.
    """
    
    # Allocate an array for predictions:
    yf = np.zeros(len(xf))
    
    for i, x in enumerate(xf):
        # For each point we want a prediction for, do a
        # nearest neighbours estimate as above:
        dists = np.abs(X - x)
        idx = np.argsort(dists)[:k]
        yf[i] = np.mean(y[idx])
        
    return yf

With this function, it is easy to compute the $k$-NN estimate for a long range of points that we can then draw a line through.

In [ ]:
# Create input points and compute outputs:
xf = np.linspace(0, 10, 200)
yf = knn(3, xf, X, y)

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X, y)
ax.plot(xf, yf, ls="-", color="C1")
plt.show()

## Cross validation

To implement $K$-fold cross validation, we need to identify $K$ random splits of (roughly) equal size. 

In [ ]:
# Number of folds to use
K = 3

fold_size = len(X)//K

# Create indices and shuffle them
idx = np.arange(len(X))
np.random.shuffle(idx)

# Divide list of indices into chunks of size fold_size
folds = []
for i in range(K):
    folds.append(idx[i * fold_size:(i + 1) * fold_size])

folds = np.array(folds)
print("Indices for each fold:\n", folds)

We can now use these indices to construct a training set and a test set and test our model:

In [ ]:
for f, fold in enumerate(folds):
    # Get indices for which folds to use for training:
    train_idx = np.delete(np.arange(K, dtype=int), f)
    # Construct training set by concatenating the indices:
    train_folds = np.concatenate(folds[train_idx])
    
    # Select data for training and tes
    X_train = X[train_folds]
    y_train = y[train_folds]
    X_test = X[fold]
    y_test = y[fold]

    # Get predictions for each test point:
    yp = knn(k, X_test, X_train, y_train)

    # Compute the RMSE
    RMSE = np.sqrt(np.mean(np.array(yp - y_test)**2))
    
    print("Fold {}: RMSE = {:g}".format(f, RMSE))

### Using CV to determine the optimal $k$

We can now use such a CV to test how many neighbours would be the optimal choice for our $k$-NN and the current dataset.

In [ ]:
CV_err = []    # list to store the mean RMSE for each k
k_range = np.arange(1,20, dtype=int)

for k in k_range:
    rmses = []
    
    for f, fold in enumerate(folds):
        # Get indices for which folds to use for training:
        train_idx = np.delete(np.arange(K, dtype=int), f)
        # Construct training set by concatenating the indices:
        train_folds = np.concatenate(folds[train_idx])

        # Select data for training and tes
        X_train = X[train_folds]
        y_train = y[train_folds]
        X_test = X[fold]
        y_test = y[fold]

        # Get predictions for each test point:
        yp = knn(k, X_test, X_train, y_train)

        # Compute the RMSE and store it
        RMSE = np.sqrt(np.mean(np.array(yp - y_test)**2))
        rmses.append(RMSE)

    print("k = {}: mean RMSE = {:g}".format(k, np.mean(rmses)))

    # Save mean and std of the RMSE for each k
    CV_err.append([np.mean(rmses), np.std(rmses)])
CV_err = np.array(CV_err)

From this, we can now locate the $k$ with the smalles RMSE and use that for further predictions on this dataset.

In [ ]:
idx_of_lowest_RMSE = np.argmin(CV_err[:,0])

print("Smallest RMSE of {:g} is achieved for k = {}.".format(CV_err[idx_of_lowest_RMSE,0], 
                                                             k_range[idx_of_lowest_RMSE]))

Finally, let's visualise the mean and std of the RMSE for each $k$:

In [ ]:
fig, ax = plt.subplots()
ax.errorbar(k_range, CV_err[:,0], yerr=CV_err[:,1])

ax.set_xticks(k_range)   # Make tick marks for all k
ax.set_xlabel("$k$")
ax.set_ylabel("RMSE")
plt.grid()
plt.show()

As seen in the plot, the best $k$ lies at the very low end of the range, but the uncertainties are also too large to make a conclusive decision on what $k$ to use. In such a situation, one might try to reduce the uncertainties by acquiring more data or increasing the number of folds in the CV. In the end, one always picks the $k$ with the smallest mean RMSE, regardless of large the uncertainties are.